In [ ]:
###############################################################################
import aliases

In [ ]:
from everest.ptolemaic.schema import Schema

In [ ]:
from everest.ptolemaic.fundaments.intt import Intt

In [ ]:
class MySchema(metaclass=Schema):

    a: Intt
    b: Intt

In [ ]:
MySchema(1, 2)

In [ ]:
MySchema[1, 2]

In [ ]:
MySchema[0:, 1:]

In [ ]:
MySchema.__incision_manager__

In [ ]:
brace = Brace[{'sig': Slc[::Brace[{'a': Slc[::Floatt], 'b': Slc[::Floatt]}]], 'foo': Slc[::Floatt]}]

In [ ]:
brace

In [ ]:
Brace[dict(a=1, b=2)]

In [ ]:
from everest.ptolemaic.fundaments.floatt import Floatt
from everest.ptolemaic.fundaments.brace import Brace
from everest.utilities import Slc
from everest.models.ode import *


@ODEModel
def Lorenz(
        t,
        state: Brace[dict(x=Slc[::Floatt], y=Slc[::Floatt], z=Slc[::Floatt])],
        sigma: Floatt = 10.,
        rho: Floatt = 28.,
        beta: Floatt = 2.667,
        /):
    x, y, z = state
    return (
        sigma * (y - x),
        rho * x - y - x * z,
        x * y - beta * z,
        )

In [ ]:
multispace = Brace[dict(
    par=Slc[::Lorenz.sig.__incision_manager__],
    state=Slc[::Lorenz.statespace],
    index=Slc[::Floatt[0.:]],
    )]
multispace

In [ ]:
multispace[..., (0., 0., 0.), 0.][0., 1.]

In [ ]:
mybrace = multispace[(0., 0., 0.), (1., 1., 1.), 2.]

In [ ]:
mybrace.state